In [57]:
%run imports.ipynb

imports loaded.


### Func def

#### pre-processing

In [2]:
print('\nPre-Processing func:')
print('********************')


Pre-Processing func:
********************


In [3]:
def get_height(item):
    
    '''return height in cm'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return round(30.48 * float(item[0:item.find("'")]) + 2.54 * float(item[item.find("'")+1:item.find('"')]),2)

print('    get_height(item)')    

    get_height(item)


In [4]:
def get_weight(item):
    
    '''return weight in kg'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return 0.45 * float(item[0:item.find('lbs.')])
    
print('    get_weight(item)')    

    get_weight(item)


In [5]:
def get_reach(item):
    
    '''return reach in cm'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return 2.54 * float(item[0:item.find('"')])
    
print('    get_reach(item)')    

    get_reach(item)


In [6]:
def get_std_reach(item, height, reach_list, height_list):
    '''return "standard reach" ( =mean(reach/height)*height )'''
    return round(height*np.mean(reach_list/height_list),2)

print('    get_std_reach(item, height, reach_list, height_list)')

    get_std_reach(item, height, reach_list, height_list)


In [7]:
def get_age(item):
    
    '''return age (instead of date of birth)'''
    
    from datetime import date,timedelta
    
    item = str(item)
    current_date = date.today().year
    
    if item[0] == '-':
        return None
    else:
        return (current_date - int(item[item.find(',')+1:]))
    
print('    get_age(item)')    

    get_age(item)


In [8]:
def get_std_age(item, age_list):
    
    '''return standrd age per weight class'''  
    return round(np.mean(age_list),2)

print('    get_std_age(item, age_list)')

    get_std_age(item, age_list)


In [9]:
def get_win_perc(name, date, df):
    
    '''return current career winning %'''
    
    wins = len(df[(df['w_fighter'] == name) & (df.date < date)])
    losses = len(df[(df['l_fighter'] == name) & (df.date < date)])
    
    if wins == 0:
        return 0
    return wins/(wins+losses)

print('    get_win_perc(name, date, df)')

    get_win_perc(name, date, df)


In [10]:
def get_lose_streak_data_frame(name, date, df):
    
    '''return losses data frame of: sequence(from last to first), result(win/lose) & opponent '''
    
    lose_streak = pd.DataFrame({
        'result': np.array('lose'),
        'date': pd.to_datetime(np.array(df.date[df.l_fighter == name])),
        'opponent': np.array(df['w_fighter'][df.l_fighter == name])
    })
    
    return lose_streak[lose_streak.date < date]

print('    get_lose_streak_data_frame(name, date, df)')

    get_lose_streak_data_frame(name, date, df)


In [11]:
def get_win_streak_data_frame(name, date, df):
    
    '''return wins data frame of: sequence(from last to first), result(win/lose) & opponent '''
    
    win_streak = pd.DataFrame({
        'result': np.array('win'),
        'date': pd.to_datetime(np.array(df.date[df.w_fighter == name])),
        'opponent': np.array(df['l_fighter'][df.w_fighter == name])
    })
    
    return win_streak[win_streak.date < date]

print('    get_win_streak_data_frame(name, date, df)')

    get_win_streak_data_frame(name, date, df)


In [12]:
def get_streak_data_frame(name, date, df):
    
    '''return losses & winns data frame sorted by date'''
    
    wins_df = get_win_streak_data_frame(name, date, df)
    losses_df = get_lose_streak_data_frame(name, date, df)
    
    return (pd.concat([wins_df, losses_df])).sort_values(by='date', ascending=False)

print('    get_streak_data_frame(name, date, df)')

    get_streak_data_frame(name, date, df)


In [13]:
def get_win_streak(name, date, df):
    
    '''return int of fighter recent win streak'''
    
    streak = get_streak_data_frame(name, date, df)
    
    count = 0
    for result in streak['result']:
        if result == 'lose':
            return count
        count += 1
    
    return count

print('    get_win_streak(name, date, df)')

    get_win_streak(name, date, df)


In [14]:
def get_lose_streak(name, date, df):
    
    '''return int of fighter recent losses streak'''
    
    streak = get_streak_data_frame(name, date, df)
    
    count = 0
    for result in streak['result']:
        if result == 'win':
            return count
        count += 1
    
    return count

print('    get_lose_streak(name, date, df)')

    get_lose_streak(name, date, df)


In [15]:
def get_fighter_stats(name, data ):
    '''return fighters statistics from stat_df'''
    
    stat_list = data.columns[1:]
    row = (data[stat_list][data.name==name]).reset_index(drop=True)
    
    if len(row) < 1:
        return np.array(len(stat_list)*[None])
    else:
        return np.array(row.loc[0])
    
print('    get_fighter_stats(name, data )')    

    get_fighter_stats(name, data )


In [16]:
def add_columns(data, column_list):
    
    '''add new columns to data '''
    
    for column in column_list:
        data[column] = None
    
    return data

print('    add_columns(data, column_list)')

    add_columns(data, column_list)


#### feature_engineering

In [17]:
print('\nFeature Engineering func:')
print('*************************')


Feature Engineering func:
*************************


In [18]:
def get_stat(index, data, stat='height', fo=''):
    return data.loc[index, fo + stat]

print("    get_stat(index, data, stat='height', fo='')")

    get_stat(index, data, stat='height', fo='')


In [19]:
def get_swaped_row(index, data):
    
    '''return row with opponent as winner and statistics swapped'''
    
    swapped_row = np.array([])
    
    for column in data.columns:
        
        if column.find('f_') > -1: # f_ will get opponent stats
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat=column[2:], fo='o_'))
        elif column.find('o_') > -1: # o_ will get fighter stats
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat=column[2:], fo='f_'))          
        elif column == 'result':
            swapped_row = np.append(swapped_row, 'lose')
        elif column == 'fighter':
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat='opponent'))
        elif column == 'opponent':
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat='fighter'))
        else:
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat=column))
    
    return swapped_row

print('    get_swaped_row(index, data)')

    get_swaped_row(index, data)


In [20]:
def get_wins(combo, data):

    data = data[(data.result=='win') & (data.stance_combo==combo)]
    return len(data)

print('    get_wins(combo, data)')

    get_wins(combo, data)


In [21]:
def get_wins_ratio(combo, data):
    
    combo_df = data[data.stance_combo==combo]
    if len(combo_df) == 0:
        return 0
    else:
        return get_wins(combo, data=data)/len(combo_df)
    
print('    get_wins_ratio(combo, data)')    

    get_wins_ratio(combo, data)


#### fight_stats_model_training

In [22]:
print('\nfight_stats_model_training func:')
print('********************************')


fight_stats_model_training func:
********************************


In [23]:
def get_sig_features(data, sig_r):
    '''return list of most significant columns from corr data frame'''
    sig_features = []
    for i in data.columns:
        for j in data.index.values:
            if data.loc[j,i] in sig_r:
                sig_features.append(j)
            
    return sig_features

print('    get_sig_features(data, sig_r)')

    get_sig_features(data, sig_r)


In [24]:
def get_r_score(X, y):
    from scipy.stats import pearsonr
    r, _ = pearsonr(X, y)
    return r

print('    get_r_score(X, y)')

    get_r_score(X, y)


In [25]:
def set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3):

    if 'set' not in data.columns:
        data.insert(0, 'set', None) # inserd 'set' column to position 0
    
    for i in range(len(df)):
        
        if data.loc[i,'result'] == None:
            data.loc[i,'set'] = 'new'
        
        elif random.random() < test_size:
            data.loc[i,'set'] = 'test'
        else:
            if random.random() < valid_size:
                data.loc[i,'set'] = 'validation'
            else:
                data.loc[i,'set'] = 'train' 

    X_train = data[X_columns][data.set=='train'] 
    X_test = data[X_columns][data.set=='test']
    y_train = data[y_columns][data.set=='train']
    y_test = data[y_columns][data.set=='test']
    X_val = data[X_columns][data.set=='validation']
    y_val = data[y_columns][data.set=='validation']

    print('Data split: Data=%s, train_set=%s, validation_set=%s, test_set=%s'  %(str(data.shape), str(X_train.shape), str(X_val.shape), str(X_test.shape)))
    return data

print('    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)')

    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)


In [26]:
def set_knn_reg(X_columns, y_columns, data, n=5):
    
    knn_reg = KNeighborsRegressor(n_neighbors=n)
    X_train = data[X_columns][data.set=='train']
    y_train = data[y_columns][data.set=='train']
    knn_reg.fit(X_train,y_train)
    return knn_reg
    
print('    set_knn_reg(X_columns, y_columns, data, n=5)')    

    set_knn_reg(X_columns, y_columns, data, n=5)


In [27]:
def set_predictions(model, data, X_columns, y_columns):
    
    for column in y_columns:
        data[column + '_pred'] = None
        data[column + '_pred'] = model.predict(data[X_columns])
    
    return data

print('    set_predictions(model, data, X_columns, y_columns)')

    set_predictions(model, data, X_columns, y_columns)


In [28]:
def get_rmse(act, pred):
    return round(math.sqrt(mean_squared_error(act, pred))/(np.max(act) - np.min(act)),4)

print('    get_rmse(act, pred)')

    get_rmse(act, pred)


In [29]:
def get_grad_descent_step(alpha, X_columns, y_columns, data, n):
    row = {}
    row['neighbors'] = [n]
    
    knn_model = set_knn_reg(X_columns=X_columns, y_columns=y_columns, data=data, n=n)
    data = set_predictions(model=knn_model, X_columns=X_columns, y_columns=y_columns, data=data)
    
    total_rmse = 0
    for stat in ['kd','str','td','sub']:
        row[stat + '_dif rmse'] = [get_rmse(act=df[stat+'_dif'][df.set=='validation'], pred=df[stat+'_dif_pred'][df.set=='validation'])]
        total_rmse += np.mean(row[stat + '_dif rmse'])
        
    row['total_rmse'] = np.round(total_rmse, 4)
    return pd.DataFrame.from_dict(data=row)

print('    get_grad_descent_step(alpha, X_columns, y_columns, data, n)')    

    get_grad_descent_step(alpha, X_columns, y_columns, data, n)


#### fight_result_model_training

In [30]:
print('\nfight_result_model_training func:')
print('*********************************')


fight_result_model_training func:
*********************************


In [66]:
def set_knn_clf(X_columns, y_columns, data, neighbors=3):
    
    knn_clf = KNeighborsClassifier(n_neighbors=neighbors)
    X_train = data[X_columns][data.set=='train']
    y_train = np.ravel(np.array(data[y_columns][data.set=='train']).reshape(-1,1))
    knn_clf.fit(X_train,y_train)
    return knn_clf

print('    set_knn_clf(X_columns, y_columns, data, neighbors=3)')

    set_knn_clf(X_columns, y_columns, data, neighbors=3)


In [70]:
def get_knn_grad_step( X_columns, y_columns, data, neighbors):
    
    knn_clf = set_knn_clf(X_columns, y_columns, data, neighbors=neighbors)
    data = set_predictions(model=knn_clf, data=data, X_columns=X_columns, y_columns=y_columns)
    
    row = {'neighbors':n}
    row['accuracy'] = [get_accuracy(act=data.result[data.set=='validation'], pred=data.result_pred[data.set=='validation'])]
    
    return pd.DataFrame.from_dict(data=row)

print('    get_knn_grad_step( X_columns, y_columns, data, neighbors)')

    get_knn_grad_step( X_columns, y_columns, data, neighbors)


In [32]:
def get_model_evaluation(data):

    y_test_act = data['result'][data.set=='test']
    y_test_pred = data['result_pred'][data.set=='test']

    print('Accuracy = %.6f' % accuracy_score(y_test_act,y_test_pred))
    ConfusionMatrixDisplay.from_predictions(y_test_act,y_test_pred)  
    
print('    get_model_evaluation(data)')    

    get_model_evaluation(data)


In [33]:
def get_accuracy(act, pred):
    return accuracy_score(act, pred)

print('    get_accuracy(act, pred)')

    get_accuracy(act, pred)


In [75]:
def get_knn_clf_param(filename):
    '''return knn_clf parameters from file'''
    data = pd.read_csv(filename).drop(columns='Unnamed: 0')
    data = data[data.ml_model=='knn_clf']
    return np.min(data.value)

print('    get_knn_clf_param(filename)')

    get_knn_clf_param(filename)


#### predictor

In [36]:
print('\nPrediction func:')
print('****************')


Prediction func:
****************


In [37]:
def get_current_date():
    return pd.to_datetime(date.today())

print('    get_current_date()')

    get_current_date()


In [38]:
def get_win_perc(fighter, data):
    '''return current career winning %'''
    date = get_current_date()
    wins = len(data[(data['fighter'] == fighter)&(data.date < date) & (data['result']=='win')]) + len(data[(data['opponent'] == fighter)&(data.date < date) & (data['result']=='lose')])
    losses = len(data[(data['fighter'] == fighter)&(data.date < date) & (data['result']=='lose')]) + len(data[(data['opponent'] == fighter)&(data.date < date) & (data['result']=='win')])
    return wins
    
    if wins == 0:
        return 0
    return wins/(wins+losses)

print('    get_win_perc(fighter, data)')

    get_win_perc(fighter, data)


In [39]:
def get_lose_streak(fighter, data):
    '''return current lose streak'''
    date = get_current_date()
    data = ((data[(data.date < date)&((data.fighter == fighter)|(data.opponent == fighter))]).sort_values(by=['date'], ascending=False)).reset_index(drop=True)
    
    count = 0
    for i in range(0,len(data)):
        
        result = data.loc[i,'result']
        f = data.loc[i,'fighter']
        o = data.loc[i,'opponent']
        
        if ((result == 'lose')&(f == fighter))|((result == 'win')&(o == fighter)):
            count += 1
        else:
            return count
    return count

print('    get_lose_streak(fighter, data)')

    get_lose_streak(fighter, data)


In [40]:
def get_win_streak(fighter, data):
    '''return current win streak'''
    date = get_current_date()
    data = ((data[(data.date < date)&((data.fighter == fighter)|(data.opponent == fighter))]).sort_values(by=['date'], ascending=False)).reset_index(drop=True)
    
    count = 0
    for i in range(0,len(data)):
        
        result = data.loc[i,'result']
        f = data.loc[i,'fighter']
        o = data.loc[i,'opponent']
        
        if ((result == 'win')&(f == fighter))|((result == 'lose')&(o == fighter)):
            count += 1
        else:
            return count
    return count

print('    get_win_streak(fighter, data)')

    get_win_streak(fighter, data)


In [41]:
def get_td_avg(fighter, data):
    '''return current takedown average'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'td avg.']

print('    get_td_avg(fighter, data)')

    get_td_avg(fighter, data)


In [42]:
def get_td_acc(fighter, data):
    '''return current takedown accuracy'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'td acc.']

print('    get_td_acc(fighter, data)')

    get_td_acc(fighter, data)


In [43]:
def get_td_def(fighter, data):
    '''return current takedown defense'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'td def.']

print('    get_td_def(fighter, data)')

    get_td_def(fighter, data)


In [44]:
def get_str_acc(fighter, data):
    '''return current striking accuracy'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'str. acc.']

print('    get_str_acc(fighter, data)')

    get_str_acc(fighter, data)


In [45]:
def get_str_def(fighter, data):
    '''return current striking defense'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'str. def']

print('    get_str_def(fighter, data)')

    get_str_def(fighter, data)


In [46]:
def get_sub_avg(fighter, data):
    '''return current submission average'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'sub. avg.']

print('    get_sub_avg(fighter, data)')

    get_sub_avg(fighter, data)


In [47]:
def get_sapm(fighter, data):
    '''return current strikes absorbed per minute'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'sapm']

print('    get_sapm(fighter, data)')

    get_sapm(fighter, data)


In [48]:
def get_slpm(fighter, data):
    '''return current strikes landed per minutes'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'slpm']

print('    get_slpm(fighter, data)')

    get_slpm(fighter, data)


In [49]:
def get_stance(fighter, data):
    '''return fighter stance'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'stance']

print('    get_stance(fighter, data)')

    get_stance(fighter, data)


In [50]:
def get_stance_wins_ratio(fighter, data):
    
    stance = get_stance(fighter=fighter, data=data)
    data = data[data.stance==stance].reset_index(drop=True)
    wins = np.sum(data.wins)
    losses = np.sum(data.losses)
    return wins/(wins+losses)

print('    get_stance_wins_ratio(fighter, data)')

    get_stance_wins_ratio(fighter, data)


In [51]:
def set_new_fight(fighter, opponent, data):
    
    row = {'set': None,
           'date': get_current_date(),
           'result': None,
           'fighter': fighter,
           'opponent': opponent,
           'kd_dif':None,
           'str_dif':None,
           'td_dif':None,
           'sub_dif':None,
           
           'td avg._dif': get_td_avg(fighter=fighter, data=stats) - get_td_avg(fighter=opponent, data=stats),
           'td acc._dif': get_td_acc(fighter=fighter, data=stats) - get_td_acc(fighter=opponent, data=stats),
           'td def._dif': get_td_def(fighter=fighter, data=stats) - get_td_def(fighter=opponent, data=stats),
           'sub. avg._dif': get_sub_avg(fighter=fighter, data=stats) - get_sub_avg(fighter=opponent, data=stats),
           'slpm_dif': get_slpm(fighter=fighter, data=stats) - get_slpm(fighter=opponent, data=stats),
           'str. acc._dif': get_str_acc(fighter=fighter, data=stats) - get_str_acc(fighter=opponent, data=stats),
           'sapm_dif': get_sapm(fighter=fighter, data=stats) - get_sapm(fighter=opponent, data=stats),
           'str. def_dif': get_str_def(fighter=fighter, data=stats) - get_str_def(fighter=opponent, data=stats),
           
           'stance_win_ratio':get_stance_wins_ratio(fighter=fighter, data=stats) - get_stance_wins_ratio(fighter=opponent, data=stats),
           'win_perc_dif': get_win_perc(fighter=fighter, data=data) - get_win_perc(fighter=opponent, data=data),
           'win_strk_dif': get_win_streak(fighter=fighter, data=data) - get_win_streak(fighter=opponent, data=data),
           'lose_strk_dif': get_lose_streak(fighter=fighter, data=data) - get_lose_streak(fighter=opponent, data=data),         
          }
    return pd.DataFrame(row, index=[0])

print('    set_new_fight(fighter, opponent, data)')

    set_new_fight(fighter, opponent, data)


In [52]:
def set_knn_predictor(X_columns, y_columns, data, n=5):
    
    knn_reg = KNeighborsRegressor(n_neighbors=n)
    X_train = data[X_columns][data.set!='new']
    y_train = data[y_columns][data.set!='new']
    knn_reg.fit(X_train,y_train)
    return knn_reg
    
print('    set_knn_predictor(X_columns, y_columns, data, n=5)')  

    set_knn_predictor(X_columns, y_columns, data, n=5)


In [53]:
def set_knn_predictions(model, data, X_columns, y_columns):
    
    for column in y_columns:
        data[column] = None
        data[column] = model.predict(data[X_columns])
    
    return data

print('    set_knn_predictions(model, data, X_columns, y_columns)')

    set_knn_predictions(model, data, X_columns, y_columns)


In [76]:
def set_knn_clf_predictor(X_columns, y_columns, data, neighbors):
    
    knn_clf = KNeighborsClassifier(n_neighbors=n)
    X_train = data[X_columns][data.set!='new']
    y_train = data[y_columns][data.set!='new']
    
    if len(y_columns) == 1:
        y_train = np.ravel(np.array(data[y_columns[0]][data.set!='new']).reshape(-1,1))
    else:    
        y_train = data[y_columns][data.set!='new']
    
    knn_clf.fit(X_train,y_train)
    return knn_clf

print("    set_knn_clf_predictor(X_columns, y_columns, data, neighbors)")        

    set_knn_clf_predictor(X_columns, y_columns, data, neighbors)


In [78]:
def set_predictions(model, data, X_columns, y_columns):
    
    for column in y_columns:
        data[column + '_pred'] = None
        data[column + '_pred'] = model.predict(data[X_columns])
    
    return data

print('    set_predictions(model, data, X_columns, y_columns)')

    set_predictions(model, data, X_columns, y_columns)


In [55]:
>>>>

SyntaxError: invalid syntax (<ipython-input-55-d960544c3de0>, line 1)

In [ ]:
def get_fighters_stats(fighter, opponent, data):
    
    data1 = pd.DataFrame((data[data.name == fighter]).reset_index(drop=True).loc[0])[1:].rename(columns={0:fighter})
    data2 = pd.DataFrame((data[data.name == opponent]).reset_index(drop=True).loc[0])[1:].rename(columns={0:opponent})
    return pd.concat([data1,data2], axis=1)

print('    get_fighters_stats(fighter, opponent, data)')

In [ ]:
>>>>>>>>>>>>

In [ ]:
def get_winner(fighter, opponent, data):
    '''return winner of the fight'''
    winner = None
    data = (data[(data['fighter'] == fighter) & (data['opponent'] == opponent)]).reset_index(drop=True)
    
    if data.loc[0,'result_pred'] == 'win':
        winner = fighter
    else: 
        winner = opponent    
    
    return winner

print('get_winner(fighter=fighter, opponent, data)')